# Q1
* answer: 0.1.48

In [2]:
!docker exec -it ollama ollama -v

ollama version is 0.1.48


In [4]:
!docker info

Client:
 Version:    26.1.3-1
 Context:    default
 Debug Mode: false
 Plugins:
  buildx: Docker Buildx (Docker Inc.)
    Version:  v0.14.1
    Path:     /usr/libexec/docker/cli-plugins/docker-buildx
  compose: Docker Compose (Docker Inc.)
    Version:  v2.27.1
    Path:     /usr/libexec/docker/cli-plugins/docker-compose

Server:
 Containers: 2
  Running: 1
  Paused: 0
  Stopped: 1
 Images: 3
 Server Version: 26.1.3-1
 Storage Driver: overlay2
  Backing Filesystem: extfs
  Supports d_type: true
  Using metacopy: false
  Native Overlay Diff: false
  userxattr: false
 Logging Driver: json-file
 Cgroup Driver: cgroupfs
 Cgroup Version: 2
 Plugins:
  Volume: local
  Network: bridge host ipvlan macvlan null overlay
  Log: awslogs fluentd gcplogs gelf journald json-file local splunk syslog
 Swarm: inactive
 Runtimes: io.containerd.runc.v2 runc
 Default Runtime: runc
 Init Binary: docker-init
 containerd version: 8b3b7ca2e5ce38e8f31a34f35b2b68ceb8470d89
 runc version: 51d5e94601ceffbbd85688df

# Q2

* Join the container: Enter the running ollama container:
```bash
docker exec -it ollama /bin/bash
```
* Pull the model: 
```bash
ollama pull gemma:2b
```
* CD to a library: 
```bash
cd /root/.ollama/models/manifests/registry.ollama.ai/library
```
* LS
```bash
ls
```
* cat file:
```bash
cat gemma/2b
```

Output:

{"schemaVersion":2,"mediaType":"application/vnd.docker.distribution.manifest.v2+json","config":{"mediaType":"application/vnd.docker.container.image.v1+json","digest":"sha256:887433b89a901c156f7e6944442f3c9e57f3c55d6ed52042cbb7303aea994290","size":483},"layers":[{"mediaType":"application/vnd.ollama.image.model","digest":"sha256:c1864a5eb19305c40519da12cc543519e48a0697ecd30e15d5ac228644957d12","size":1678447520},{"mediaType":"application/vnd.ollama.image.license","digest":"sha256:097a36493f718248845233af1d3fefe7a303f864fae13bc31a3a9704229378ca","size":8433},{"mediaType":"application/vnd.ollama.image.template","digest":"sha256:109037bec39c0becc8221222ae23557559bc594290945a2c4221ab4f303b8871","size":136},{"mediaType":"application/vnd.ollama.image.params","digest":"sha256:22a838ceb7fb22755a3b0ae9b4eadde629d19be1f651f73efb8c6b4e2cd0eea0","size":84}]}

# Q3

* Join the container: Enter the running ollama container:
```bash 
docker exec -it ollama /bin/bash
```
* Run the model 
```bash 
ollama run gemma2`
```

* Make the prompt: 
```bash 
 >>> "10 * 10"` 
```

'Output:' : "10 * 10 = 100"


Model Info: 
```bash
>>> /show info
```
Output: 

  Model                             
        arch                    gemma2              
        parameters              9.2B                
        quantization            Q4_0                
        context length          8192                
        embedding length        3584                
                                          
  Parameters                        
        num_ctx         4096                       
        repeat_penalty  1                          
        stop            "<start_of_turn>"          
        stop            "<end_of_turn>"            
                                          
  License                           
        Gemma Terms of Use                        
        Last modified: February 21, 2024       

# Q4
* answer 1.6G (closest: 1.7G)

* 1) Creating a new dir:
```bash 
mkdir ollama_files
```
* 2) Stopping  ollama container (exisitng one):
```bash 
docker stop ollama
```

* 3) Removing ollama (exisitng prev container)
```bash 
docker rm ollama
```



* 4) Run a new container with a folder mount
```bash 
docker run -it \
    --rm \
    -v ./ollama_files:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

* 5) Pull the model again to a local folder:
```bash 
docker exec -it ollama ollama pull gemma:2b 
```

In [10]:
!du -sh ollama_files/models

1.6G	ollama_files/models


# Q5 Adding the weights


To copy the downloaded model files from the ollama_files/models directory to the new Docker image, you can use the following COPY instruction in the Dockerfile:

```Dockerfile
FROM ollama/ollama

COPY ollama_files/models /root/.ollama/models
```

# Q6 Serving it

* 6) Building a new image: 
```bash 
docker build -t ollama-gemma2b .
```

* 7) 11434 is busy - finding a container using it: 
```bash 
 docker ps -a | grep 11434
```
 => get the container id (2e6e47ba0d21)

* 8) Stopping that contrainer
```bash 
 docker stop 2e6e47ba0d21
```

* 9) Running a new docker image:
```bash 
docker run -it --rm -p 11434:11434 ollama-gemma2b
```

In [24]:
!curl -X GET http://localhost:11434/v1/models/list

404 page not found

* OpenAI Compatibility:
https://ollama.com/blog/openai-compatibility

In [17]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [29]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        temperature= 0.0,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response

In [30]:
prompt = "What's the formula for energy?"

response = llm(prompt=prompt)

In [33]:
response.choices[0].message.content

"Sure, here's the formula for energy:\n\n**E = K + U**\n\nWhere:\n\n* **E** is the energy in joules (J)\n* **K** is the kinetic energy in joules (J)\n* **U** is the potential energy in joules (J)\n\n**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:\n\n**K = 1/2 * m * v^2**\n\n**Potential energy (U)** is the energy an object possesses when it is in a position or has a specific configuration. It is calculated as the product of an object's mass and the gravitational constant (g) multiplied by the height or distance of the object from a reference point.\n\n**Gravitational potential energy (U)** is given by the formula:\n\n**U = mgh**\n\nWhere:\n\n* **m** is the mass of the object in kilograms (kg)\n* **g** is the acceleration due to gravity in meters per second squared (m/s^2)\n* **h** is the height or distance of the object in meters (m)\n\nThe formula for energ

In [34]:
response.usage

CompletionUsage(completion_tokens=304, prompt_tokens=0, total_tokens=304)

In [35]:
response.usage.completion_tokens

304